In [65]:
import pandas as pd
import uuid

In [66]:
!pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
#Crear dimensiones base para variable a nivel sujeto
temaInteres = pd.DataFrame(columns=['id','nombre','descripcion'])

etapa= pd.DataFrame(columns=['id','numEtapa','nombreAnalisis','nombreBD','descripcion',
                             'eventoInicial','eventoFinal'])

fecha=pd.DataFrame(columns=['dia','mes','anio'])

evento=pd.DataFrame(columns=['id','nombre','descripcion'])

variable=pd.DataFrame(columns=['id','nombreBD','nombreAnalisis','unidades','edadCorregida','derivada',
                               'impacto','tipoDato','tipoVariable','nivelMedicion','longitudinal',
                               'descripcionCorta','descripcionLarga'])


In [68]:
# Conseguir dataframes de diccionario
file_dict=pd.ExcelFile('data/KMC-diccionario.xlsx')
df_etapas=pd.read_excel(file_dict, sheet_name='ETAPAS')
df_temas=pd.read_excel(file_dict, sheet_name='Temas de Interés')
df_variables=pd.read_excel(file_dict, sheet_name='KMC-12mEC-DIC-VAR-version2024')
df_variables = df_variables.dropna(subset=['VAR-ID'])

In [69]:
#Poblar dimensión de eventos
eventos = pd.unique(
    pd.concat([
        df_etapas['Evento inicial (id-var)'],
        df_etapas['Evento-final (id-var)']
    ])
).tolist()

ids_eventos = [str(uuid.uuid4()) for _ in range(len(eventos))]

for i in range(len(eventos)):
    evento.loc[len(evento)] = [
        ids_eventos[i],
        eventos[i],
        ""
    ]

In [70]:
#Poblar dimensión de etapas
def encontrar_evento(nombre):
    fila=evento[evento['nombre']==nombre]
    return fila['id'].values[0]

ids_etapas = [str(uuid.uuid4()) for _ in range(len(df_etapas))]

for index, row in df_etapas.iterrows():
    etapa.loc[row['NUMERO ETAPA']]=[
        ids_etapas[index],
        row['NUMERO ETAPA'],
        row['ID-ETAPA'].lower(),
        row['NOMBRE CORTO'].lower(),
        row['DESCRIPCION'].lower(),
        encontrar_evento(row['Evento inicial (id-var)']),
        encontrar_evento(row['Evento-final (id-var)'])
    ]

In [71]:
#Poblar dimensión tema de interes
ids_temas = [str(uuid.uuid4()) for _ in range(len(df_temas))]

for index,row in df_temas.iterrows():
    temaInteres.loc[row['Número TdI']]=[
        ids_temas[index],
        row['ID-TdI'],
        row['DescripciOn']
    ]

In [72]:
#Poblar dimension fecha
fecha.loc[0]=[
    1,
    1,
    2025
]

In [73]:
#Poblar dimension variable

ids_vars = [str(uuid.uuid4()) for _ in range(len(df_variables))]

for index,row in df_variables.iterrows():
    if (index<634):
        variable.loc[index]=[
            ids_vars[index],
            row['NOMBRE EN LA BdeD'],
            row['VAR-ID'],
            row['UNITS'],
            False, #TODO: Edad coregida:De donde se saca esto?
            False, #TODO: Derivada: De donde se saca esto?
            False, #TODO: Impacto: ¿?
            row['VAR-TYPE'],
            False, #Tipo Variable
            False, #Nivel medicion
            False, #Longitudinal
            row['VAR-SHORT DESCRIPTION'],
            row['VAR-LONG DESCRIPTION']
        ]

In [74]:
#Crear diccionario de valor categorias
cols = [
    "VAL1", "SHORT-NAME1", "VAL2", "SHORT-NAME2",
    "VAL3", "SHORT-NAME3", "VAL4", "SHORT-NAME4", "VAL5", "SHORT-NAME5",
    "VAL6", "SHORT-NAME6", "VAL7", "SHORT-NAME7", "VAL8", "SHORT-NAME8",
    "VAL9", "SHORT-NAME9", "VAL10", "SHORT-NAME10", "VAL11", "SHORT-NAME11"
]

df_categorias=df_variables[cols]

unique_dicts = set()

def add_unique_dict(d):
    frozen = frozenset(d.items())
    if frozen not in unique_dicts:
        unique_dicts.add(frozen)

for index, row in df_categorias.iterrows():
    for i, col in  enumerate(df_categorias.columns):
        
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
            dicc={
                'value':int(row[col]),
                'description':str(row[siguiente_col]).strip()
            }
            add_unique_dict(dicc)


In [75]:
valorCategoria=pd.DataFrame(columns=['id','codificacion','descripcion'])
for d in unique_dicts:
    data = dict(d) 
    nueva_fila = {
        'id': str(uuid.uuid4()),               
        'codificacion': data['value'],         
        'descripcion': data['description']     
    }
    valorCategoria.loc[len(valorCategoria)] = nueva_fila

valorCategoria

,id,codificacion,descripcion
0,f436adca-c821-4c96-9788-bd1b6239b660,8,O+
1,0aee9117-38d7-4ca8-9ee5-63afbc670023,1,Med.Obstetra
2,098bf570-2a99-4a58-b277-d57ecd0f3106,1,SI Hosp
3,ef902a11-208f-43e8-9218-e34a3fb7fb88,1,No llegó al PMC
4,557bd7a1-ef31-4f7e-be9b-89d85a2b46b8,0,NO APNEA REP
...,...,...,...
311,614e7563-031e-428c-90c3-5f9e177d89cd,4,hierro solo
312,324b77a5-7f0c-4529-a39a-452a72b34947,3,Retraso MODERADO
313,3c3b049b-1262-422c-a517-2c40d25444e9,3,40semEG-3mEC
314,71097942-7d33-4dbe-a2bc-a4cbd599c841,3,"SI, con tratamiento (Cirugía)"


In [87]:
#Creacion tabla de hechos variable sujeto 
hechoEvaluarVariableSujeto= pd.DataFrame(columns=['idVariable','idEtapa','idGrupoValorCategoria',
                                                  'idGrupoTemaInteres','idFechaModificacion','idFechaRegistro',
                                                  'idGrupoOperacion','idGrupoVariableLongitudinal'])

In [84]:
#Retorna la lista de ids de categorias relacionadas a  una fila (variable)
def definirGrupoIdValorCategoria(row):
    row=row[cols] #solo se tiene en cuenta las columnas de la codificación de categoria
    id=[]
    for i, col in  enumerate(df_categorias.columns):           
        if i + 1 < len(df_categorias.columns):  
            siguiente_col = df_categorias.columns[i + 1]
        if pd.notna(row[col]) and 'VAL' in col and row[col] != ' ':
                val=valorCategoria[
                    (valorCategoria['codificacion']==row[col]) &
                    (valorCategoria['descripcion']==row[siguiente_col])
                ]
                if not val.empty:
                    id.append(str(val['id'].iloc[0]))
                
    return id

In [ ]:
#Poblar tabla de hechos 
bridgeCategoria=pd.DataFrame(columns=['idGrupoValorCategoria','idValorCategoria'])

ids_grupoCats = [str(uuid.uuid4()) for _ in range(len(variable))]

for index,row in df_variables.iterrows():
    #id variable
    row_var = variable[
        variable['nombreAnalisis']==
        str(row['VAR-ID'])
    ]  
    var= str(row_var['id'].iloc[0])
    
    #id etapa
    row_etapa=etapa[etapa['numEtapa']==(row['#ETAPA'])]
    id_etapa=str(row_etapa['id'].iloc[0])

    #id categoria y grupo
    grupo_categoria=ids_grupoCats[index]
    ids_valores_cats=definirGrupoIdValorCategoria(row)
    
    #Poblar dimensión bridge categoria
    for i in ids_valores_cats:
        bridgeCategoria.loc[len(bridgeCategoria)]=[
            grupo_categoria,
            i
        ]

    #id tema interes
    row_tema=temaInteres[temaInteres['nombre']==row['id-TEMA DE INTERES1']]
    if not row_tema.empty:
        id_tema = str(row_tema['id'].iloc[0])
    else: #hay 4 variables que no tienen tema de interés relacionada
        id_tema=None

    
    #id fecha modificación y registro
    fecha='1,1,2025'

    id_operacion=None #TODO: ¿como sacarlo para variables derivadas?
    id_longitudinal=None #TODO: ¿como determinar las relacionadas?
    
    hechoEvaluarVariableSujeto.loc[index]=[
        var,
        id_etapa,
        grupo_categoria,
        id_tema,
        fecha,
        fecha,
        id_operacion,
        id_longitudinal
    ]

hechoEvaluarVariableSujeto


,idVariable,idEtapa,idGrupoValorCategoria,idGrupoTemaInteres,idFechaModificacion,idFechaRegistro,idGrupoOperacion,idGrupoVariableLongitudinal
0,525a87ea-54de-4b87-b396-4ae81d4d9f0a,093d2dab-582b-49dd-8bf8-e774cdb0013d,74934acc-82ca-400a-8938-572e31fa9d96,cfea13e9-cb32-4a15-a605-b7dbc7f5adc5,"1,1,2025","1,1,2025",None,None
1,a9302a24-5f6d-4178-a2fc-8c01a120fad9,093d2dab-582b-49dd-8bf8-e774cdb0013d,7c53a936-e4f8-459d-920f-b7958ed23418,None,"1,1,2025","1,1,2025",None,None
2,d3720e2f-ee8f-4aa4-b273-71851e2f1509,093d2dab-582b-49dd-8bf8-e774cdb0013d,543b2178-5b80-4bac-a90f-abbaf342054f,None,"1,1,2025","1,1,2025",None,None
3,ce711bef-eb72-4ae4-b26d-ce34aabf7fae,093d2dab-582b-49dd-8bf8-e774cdb0013d,e1a0ec29-8e30-4807-8c81-f79a738816c2,None,"1,1,2025","1,1,2025",None,None
4,9eea66c3-1e56-4c9f-a117-622b220c0f6a,093d2dab-582b-49dd-8bf8-e774cdb0013d,4ebef667-244f-4e47-ba20-bf06801f9c62,None,"1,1,2025","1,1,2025",None,None
...,...,...,...,...,...,...,...,...
629,a6591200-99a9-41fe-8b18-3f8cc95ed28a,0579c5a7-c47f-4e3d-8b86-e64c7091b015,70be2643-7a69-4869-affc-a5a387bf28b5,cfea13e9-cb32-4a15-a605-b7dbc7f5adc5,"1,1,2025","1,1,2025",None,None
630,87567bf1-bbd2-4bda-af9c-ebf652ffe930,0579c5a7-c47f-4e3d-8b86-e64c7091b015,22603c5f-05cb-4129-88d9-bbb15180bc4d,cfea13e9-cb32-4a15-a605-b7dbc7f5adc5,"1,1,2025","1,1,2025",None,None
631,ca5a58af-c558-4927-8f17-770a0d9223f5,0579c5a7-c47f-4e3d-8b86-e64c7091b015,0a7ecf68-58ca-43fb-bda8-b9664aa17799,cfea13e9-cb32-4a15-a605-b7dbc7f5adc5,"1,1,2025","1,1,2025",None,None
632,1ec7ac64-7c43-4f14-9f4e-db0a4169b1ac,0579c5a7-c47f-4e3d-8b86-e64c7091b015,2ce63efd-a36a-447c-9828-a9bd7041ab01,cfea13e9-cb32-4a15-a605-b7dbc7f5adc5,"1,1,2025","1,1,2025",None,None


Anotaciones:
- La meta de donde sale?
- Los eventos no relacionados a la variable pero no a una etapa donde estan?
- En este momento se registraron las operaciones que se hicieron sobre las variables?
- ¿Cómo identificar variables longitudinales?
- Aun no tengo la BD  de indicadores
- Es necesario tener un grupo de temas de interés? (no hay ninguna variable en el diccionario que tenga más de un tema de interés)